In [0]:
import tensorflow as tf
import math
import numpy as np

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

# For MNIST
mnist = mnist_data.read_data_sets("MNIST_data/", one_hot= True)

# For Fashion MNIST
# mnist = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/', one_hot= True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [5]:
#==================== BASELINE CNN =========================#

# to reset the Tensorflow default graph
tf.reset_default_graph()

########################################
# define variables and placeholders
########################################

X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1), name="X")
Y = tf.placeholder(tf.float32, shape=(None, 10), name="Y")

# variable learning rate

step = tf.placeholder(tf.int32)
lr = 0.0001 +  tf.train.exponential_decay(0.005, step, 2000, 1/math.e)

pkeep = tf.placeholder(tf.float32)

should_drop = tf.placeholder(tf.bool)



########################################
# build the model
########################################

conv1 = tf.layers.conv2d(inputs=X, filters=256, kernel_size=[5, 5], strides=(1,1), padding="valid", activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), bias_initializer='zeros')
conv2 = tf.layers.conv2d(inputs=conv1, filters=256, kernel_size=[5, 5], strides=(1,1), padding="valid", activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), bias_initializer='zeros')
conv3 = tf.layers.conv2d(inputs=conv2, filters=128, kernel_size=[5, 5], strides=(1,1), padding="valid", activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), bias_initializer='zeros')

flat_dims = conv3.shape[1] * conv3.shape[2] * conv3.shape[3] 

flat = tf.reshape(conv3, [-1, flat_dims])

hidden1 = tf.layers.dense(inputs=flat, units=328, activation=tf.nn.relu)
hidden2 = tf.layers.dense(inputs=hidden1, units=192, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=hidden2, rate=0.25, training=should_drop)

output = tf.layers.dense(inputs=dropout, units=10)




########################################
# define the Loss function
########################################
cross_entropy = tf.losses.softmax_cross_entropy(Y, output)
cross_entropy = tf.reduce_mean(cross_entropy) * 100


########################################
# define accuracy
########################################
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(output, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


########################################
# training the model
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

steps = 5500 #5500*100 = 55000 * 10
with tf.Session() as sess:
  sess.run(init)
  start_time = time.time()
  for i in range(steps):
    batch_X, batch_Y = mnist.train.next_batch(100)
    
    batch_X = np.reshape(batch_X,[100,28,28,1])
    
    _,cost = sess.run([train_step,cross_entropy], feed_dict={X: batch_X, Y: batch_Y, step: i, pkeep:0.75, should_drop:True})
    
    if i%200 == 0:
      print("iteration {0}, cost is {1}".format(i, cost))
  
  end_time = time.time()
  n_test_subset = 5000
  test_images = mnist.test.images[0:n_test_subset]
  test_labels = mnist.test.labels[0:n_test_subset]
  
  print(test_images.shape)
  test_data={X: np.reshape(test_images,[n_test_subset,28,28,1]), Y: test_labels, pkeep:1.0, should_drop:False}
  
  print("Time is {}".format(end_time-start_time))
  

  acc = sess.run([accuracy], feed_dict=test_data)
  print("Accuracy on the test set is: {0}".format(acc))
  
    

iteration 0, cost is 230.46170043945312
iteration 200, cost is 18.074941635131836
iteration 400, cost is 21.10568618774414
iteration 600, cost is 5.552218914031982
iteration 800, cost is 3.175575017929077
iteration 1000, cost is 5.1382975578308105
iteration 1200, cost is 1.7702337503433228
iteration 1400, cost is 4.496580600738525
iteration 1600, cost is 1.375048279762268
iteration 1800, cost is 0.14027920365333557
iteration 2000, cost is 0.07626865059137344
iteration 2200, cost is 4.141570568084717
iteration 2400, cost is 0.15379801392555237
iteration 2600, cost is 0.3893683850765228
iteration 2800, cost is 0.41190922260284424
iteration 3000, cost is 0.6754037141799927
iteration 3200, cost is 0.9176600575447083
iteration 3400, cost is 0.6081249117851257
iteration 3600, cost is 0.3668578863143921
iteration 3800, cost is 0.0016129912110045552
iteration 4000, cost is 0.06055937707424164
iteration 4200, cost is 0.0011101235868409276
iteration 4400, cost is 0.04544683173298836
iteration 46

In [2]:
! rm -rf CapsNet-Tensorflow
! git clone https://github.com/himalayanZephyr/CapsNet-Tensorflow.git

Cloning into 'CapsNet-Tensorflow'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 587 (delta 12), reused 0 (delta 0), pack-reused 563
Receiving objects: 100% (587/587), 1.60 MiB | 1.78 MiB/s, done.
Resolving deltas: 100% (294/294), done.


In [3]:
!pip install tqdm

In [4]:
# For MNIST
!python CapsNet-Tensorflow/download_data.py

# For Fashion MNIST
# !python CapsNet-Tensorflow/download_data.py --dataset fashion-mnist --save_to data/fashion-mnist

>> Downloading train-images-idx3-ubyte.gz 100.1%
Successfully Downloaded train-images-idx3-ubyte.gz
Extracting  train-images-idx3-ubyte.gz
Successfully extracted

>> Downloading train-labels-idx1-ubyte.gz 113.5%
Successfully Downloaded train-labels-idx1-ubyte.gz
Extracting  train-labels-idx1-ubyte.gz
Successfully extracted

>> Downloading t10k-images-idx3-ubyte.gz 100.4%
Successfully Downloaded t10k-images-idx3-ubyte.gz
Extracting  t10k-images-idx3-ubyte.gz
Successfully extracted

>> Downloading t10k-labels-idx1-ubyte.gz 180.4%
Successfully Downloaded t10k-labels-idx1-ubyte.gz
Extracting  t10k-labels-idx1-ubyte.gz
Successfully extracted



In [6]:
import time
start_time = time.time()

# For MNIST
!python CapsNet-Tensorflow/main.py

# For Fashion MNIST
# !python CapsNet-Tensorflow/main.py --dataset fashion-mnist

end_time = time.time()
print("Training time is {}".format(end_time-start_time))

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

In [7]:
#====================TEST ACCURACY ========================#


# For MNIST
!python CapsNet-Tensorflow/main.py --is_training=False

# For Fashion MNIST
# !python CapsNet-Tensorflow/main.py --dataset fashion-mnist --is_training=False

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

In [8]:
#=================== Printing Accuracy ====================#

import csv

with open('results/test_acc.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
      print(row)

['test_acc']
['0.991']
